In [1]:
import numpy as np
from scipy.constants import hbar,h,e,c
from scipy.constants import elementary_charge as elc
F0 = 2.0678e-15

In [2]:
e_eff=(11.45+1)/2## for all calculations
Mhz=(2*np.pi*1e6)## divide on it to obtain value in mhz
j=0.5e-3      # critical current density nA/nm^2

In [3]:
import pandas as pd

# Q1 related parameters

In [4]:
def cap_func(Q1,Qg1,Qg2):
    return Q1+(1/Qg1+1/Qg2)**(-1)

In [5]:
def Ec(C):#fF
    return np.round((elc)**2/(2*C)/h/1e9*1e15,1) # return in MHz

In [6]:
path=r'E:\Github2.0\QCreator\designs\ThreeQubitsPlusTwoQubits\maxwell_simulations'
path=r'C:\Python\Github\QCreator\designs\Single_qubit_design_pp_qubits'

In [7]:
data=pd.read_csv(path+'\Q1_sim.csv',sep=',')# matrix should be done on pF

In [8]:
data

,Unnamed: 0,Bottom,Claw,Ground,MWLine,Top
0,Bottom,0.139790,-0.001197,-0.093560,-0.000277,-0.044760
1,Claw,-0.001197,0.130900,-0.116230,-0.000023,-0.013456
2,Ground,-0.093560,-0.116230,0.323010,-0.030567,-0.082658
3,MWLine,-0.000277,-0.000023,-0.030567,0.031050,-0.000183
4,Top,-0.044760,-0.013456,-0.082658,-0.000183,0.141060


In [9]:
q_name=''
Q1_cap_ground_top=data[q_name+'Top'][4]+data[q_name+'Top'][0]
Q1_cap_ground_bottom=data[q_name+'Bottom'][0]+data[q_name+'Bottom'][4]
Q1_cap_top_bottom=-data[q_name+'Bottom'][4]
Q1_cap=cap_func(Q1_cap_top_bottom,Q1_cap_ground_top,Q1_cap_ground_bottom)

In [10]:
Q1_cap_top_bottom,Q1_cap_ground_bottom,Q1_cap_ground_top

(0.04476, 0.09503, 0.0963)

In [11]:
Q1_cap

0.09259039251554906

In [12]:
Ec(Q1_cap)

209.2

In [50]:
def I(f,Ec):
    Ej=Ejf(f,Ec)
    I=Ej*2*np.pi/F0*h*1e18
    return I
def Ejf(f,Ec):
    Ej=(f+Ec)**2/(8*Ec)
    return Ej
def Eji(I):#nA
    return I*F0/(2*np.pi)/1e9/h/1e9
def IEj(Ej):
    return Ej/(F0/(Q2*np.pi)/1e9/h/1e9)
def f(Ej,Ec):
    return np.sqrt(8*Ej*Ec)-Ec
def Ec(C):#fF
    return elc**2/(2*C)/h/1e9*1e15

In [68]:
a1    = 0.170
a2    = 0.300 # Junction width in um
#Calculate Qubit frequency:
JJ_area = a1*a2*1e6 #in nm^2
print('Critical current density is: ',j*1e3, 'uA/um^2')
Ic = JJ_area*j #in nA
print('Critical current: ',Ic ,' nA')
print('Ej: ',Eji(Ic) ,' GHz')

print('junction area  in um: ',JJ_area*1e-6)

Critical current density is:  0.5 uA/um^2
Critical current:  25.500000000000004  nA
Ej:  12.665222203840148  GHz
junction area  in um:  0.051000000000000004


In [69]:
f(Eji(Ic),Ec(Q1_cap)/1e3)

4.394801894827204

### Resonator coupling

In [23]:
def line_qubit_coupling(Q1g,Q2g,Q1gc,Q2gc):
    return np.abs((Q1g*Q2gc-Q2g*Q1gc)/(Q1g+Q2gc+Q2g+Q1gc))

In [31]:
def cap_pretty_print(args,coeff):
    for i in args:
        print(np.round(i*coeff,1))# return in fF

In [33]:
Q1_cap_res_top=-data[q_name+'Top'][1]
Q1_cap_res_bottom=-data[q_name+'Bottom'][1]
Ccr=line_qubit_coupling(Q1_cap_ground_top,Q1_cap_ground_bottom,Q1_cap_res_top,Q1_cap_res_bottom)

In [34]:
cap_pretty_print([Q1_cap_res_top,Q1_cap_res_bottom,Ccr],1e3)

13.5
1.2
5.6


In [35]:
def qubit_res_coupling(Cc,Cq,wq,wr,Zr):# capacitances in fF, frequencies in GHz
    g= Cc*10**(-12)/2*np.sqrt((2*np.pi*wq*10**(9)*Zr/(Cq*10**(-12))))*wr*10**(9)
    return np.round(g/1e6,1)# return g/2pi in MHz

In [61]:
Zr=50
omega_r=5.5
omega_q=4.3
gqr=qubit_res_coupling(Ccr,Q1_cap,omega_q,omega_r,Zr)
gqr

59.3

### Dispersive shift

In [62]:
def dispersive_shift(gqr,wq,wr,Ec): # gqr in MHz,wq,wr in GHZ, EC in MHz and positive
    chi=gqr**2/((wq-wr)*10**(3))/(1+(wq-wr)*10**(3)/(-Ec))
    return chi # in MHz

In [63]:
chi=dispersive_shift(gqr,omega_q,omega_r,Ec(Q1_cap))
chi

-0.4350340805219836

### Readout resonator properties

In [64]:
Q=8500
k=2*np.pi*omega_r*1e9/Q 
np.round(1/k*1e9,1), np.round(k/Mhz,2)# first value in ns

(246.0, 0.65)

In [65]:
Tp=(1/k)*((omega_q-omega_r)*10**3/gqr)**2
Tp*1e6#us

100.72319032200237

### Charge line

In [41]:
Q1_cap_charge_line_top=-data[q_name+'Top'][3]
Q1_cap_charge_line_bottom=-data[q_name+'Bottom'][3]
Cc_charge=line_qubit_coupling(Q1_cap_ground_top,Q1_cap_ground_bottom,Q1_cap_charge_line_top,Q1_cap_charge_line_bottom)
cap_pretty_print([Q1_cap_charge_line_top,Q1_cap_charge_line_bottom,Cc_charge],1e6)#in aF

183.3
276.9
48.2


In [42]:
Cc_charge*1e6

48.231213739015594

In [43]:
omega_q=4.4

In [44]:
Z0=50
Tcharge=(Cc_charge+Q1_cap)/(Z0*(2*np.pi*omega_q*Cc_charge)**2)*1e-6

In [45]:
Tcharge*1e6#in us

1042.0789379473458

### Total relaxation= resonator+charge line

In [53]:
Tp*Tcharge/(Tp+Tcharge)*1e6

74.96545412923993